In [1]:
#PACKAGES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing
import requests
import json
import os
import warnings
warnings.filterwarnings("ignore")

In [9]:
# Define the API endpoint
api_url = "http://172.16.1.59:8080/v1/web/test12"

# Fetch data from the API
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    # Convert the JSON response to a Python dictionary
    data = response.json()
    
    # Create a pandas DataFrame from the data
    # Assuming the API response is a list of dictionaries
    df = pd.DataFrame(data['data'])
    
else:
    print(f"Failed to fetch data: {response.status_code}")
    
display(df)
#pd.options.display.max_colwidth = 100

ConnectionError: HTTPConnectionPool(host='172.16.1.59', port=8080): Max retries exceeded with url: /v1/web/test12 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000019CE5684920>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [25]:
file_path = "data/datadummy.json"
with open(file_path,'r') as f:
    json_data = json.load(f)
data = pd.DataFrame(json_data)

data = pd.DataFrame(json_data['data'])
print(data)

       period branch agency partno  \
0  2025-01-01    999    99P   9901   
1  2025-01-01    999    99P   9902   
2  2025-01-01    999    99Q   9903   
3  2025-01-01    888    88A   8804   
4  2025-01-01    888    88A   8805   
5  2025-01-01    999    99Q   9906   
6  2025-01-01    888    88B   8807   
7  2025-01-01    999    99Q   9908   
8  2025-01-01    999    99R   9909   
9  2025-01-01    888    88B   8810   

                                                   D  
0  [100, 105, 110, 108, 115, 120, 125, 130, 128, ...  
1  [200, 210, 220, 215, 230, 240, 245, 250, 255, ...  
2  [50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...  
3  [300, 310, 320, 315, 330, 340, 345, 350, 360, ...  
4  [500, 505, 510, 515, 520, 525, 530, 535, 540, ...  
5  [150, 155, 160, 158, 165, 170, 175, 180, 185, ...  
6  [400, 405, 410, 408, 415, 420, 425, 430, 435, ...  
7  [80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...  
8  [250, 260, 270, 265, 280, 290, 295, 300, 310, ...  
9  [600, 610, 620, 615, 630, 64

In [15]:
#PARAMETER
#WMA
weights = [0.1, 0.3, 0.6]

#EWMA
alpha_wma = 0.4

#SES & DES
alpha_ses = 0.65  # ubah nilai alpha (semakin besar semakin berat ke data terbaru)
beta_des = 0.45   # ubah nilai beta (semakin besar semakin cepat beradaptasi, kalo rendah bisa terjadi lag)

In [28]:
# get mean and standart deviation of first 12 data
data['mean_12'] = data['D'].apply(lambda x: np.mean(x[:12]))
data['std_12'] = data['D'].apply(lambda x: np.std(x[:12]))

# get upper bound from mean and std
data['ub'] = data['mean_12'] + 1.5 * data['std_12']

# limit the original data to upper bound
data['clipped_d'] = data.apply(lambda row: np.clip(row['D'][:12], 0, row['ub']).tolist(), axis=1)
display(data)

,period,branch,agency,partno,D,mean_12,std_12,ub,clipped_d
0,2025-01-01,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",121.750000,13.833082,142.499623,"[100.0, 105.0, 110.0, 108.0, 115.0, 120.0, 125..."
1,2025-01-01,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",238.333333,22.110832,271.499581,"[200.0, 210.0, 220.0, 215.0, 230.0, 240.0, 245..."
2,2025-01-01,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",72.333333,14.102797,93.487529,"[50.0, 55.0, 60.0, 58.0, 65.0, 70.0, 75.0, 80...."
3,2025-01-01,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",340.000000,23.979158,375.968736,"[300.0, 310.0, 320.0, 315.0, 330.0, 340.0, 345..."
4,2025-01-01,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",527.500000,17.260263,553.390394,"[500.0, 505.0, 510.0, 515.0, 520.0, 525.0, 530..."
5,2025-01-01,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",173.583333,15.850648,197.359306,"[150.0, 155.0, 160.0, 158.0, 165.0, 170.0, 175..."
6,2025-01-01,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",423.583333,15.850648,447.359306,"[400.0, 405.0, 410.0, 408.0, 415.0, 420.0, 425..."
7,2025-01-01,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",103.583333,15.850648,127.359306,"[80.0, 85.0, 90.0, 88.0, 95.0, 100.0, 105.0, 1..."
8,2025-01-01,999,99R,9909,"[250, 260, 270, 265, 280, 290, 295, 300, 310, ...",290.000000,23.979158,325.968736,"[250.0, 260.0, 270.0, 265.0, 280.0, 290.0, 295..."
9,2025-01-01,888,88B,8810,"[600, 610, 620, 615, 630, 640, 645, 650, 660, ...",640.000000,23.979158,675.968736,"[600.0, 610.0, 620.0, 615.0, 630.0, 640.0, 645..."


In [ ]:
# Calculate Simple Moving Average
def calculate_ma(list):
    oldData = []
    newData = []
    for i in list:
        # store calculated data to old list
        oldData.append(i)
        newData.append(np.mean(oldData))
    return newData

data['ma'] = data['clipped_d'].apply(calculate_ma)
data['ma_result'] = data['ma'].apply(lambda x: x[-1:])
data['ma_result'] = data['clipped_d'].apply(lambda x: np.mean(x))
display(data)


In [49]:
# Calculate Exponential Weighted Moving Average (EWMA)
def calculate_ewma(values, alpha):
    ewma_values = []
    previous_ewma = None
    for value in values:
        if previous_ewma is None:
            ewma_t = value if not np.isnan(value) else 0  # Initialize with the first value
        elif np.isnan(value):
            ewma_t = previous_ewma  # Skip updating when value is NaN
        else:
            ewma_t = alpha * value + (1 - alpha) * previous_ewma
        ewma_values.append(ewma_t)
        previous_ewma = ewma_t
    return ewma_values

# Apply EWMA to each row
data['ewma'] = data['clipped_d'].apply(lambda x: calculate_ewma(x, alpha_wma))
data['ewma_result'] = data['ewma'].apply(lambda x: x[-1] if x else None)

if data.shape[1] == 13:
    last_ewma = data['ewma'].iloc[-1][-1] if data['ewma'].iloc[-1] else None


TypeError: 'float' object is not iterable